In [1]:
#Import necessary Libraries
import os
import re
import rasterio
import fiona
import geopandas as gpd
import numpy as np
from PIL import Image
from numpy import array
from shapely.geometry import shape
from rasterio.windows import Window, from_bounds
from rasterio.mask import mask
from tqdm.notebook import tqdm


In [2]:
# Define the input TIFF file path
input_tiff ="1.Dataset/Enschede/Inference/ENS_RGB_008.tif"

# Define the input shapefile Bbox path
input_bbox = "1.Dataset/Enschede/Inference/Outlines_Buildings_Buffer2m.shp"

# Define the input shapefile innerplanes path
input_planes = "1.Dataset/Enschede/Inference/Inner_Buildings_Planes.shp"

# Define the output folder path
output_folder = "2.TrainingPreparation/EnschedeInference"

In [3]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
if not os.path.exists(output_folder):
    os.mkdir(output_folder+"/npy")
    os.mkdir(output_folder+"/geom_txt")
    os.mkdir(output_folder+"/clipped")
    os.mkdir(output_folder+"/rgb")
    os.mkdir(output_folder+"/inner_planes")
    os.mkdir(output_folder+"/annot")
    os.mkdir(output_folder+"/det_final")

In [4]:
# Open the TIFF file and get its spatial reference
ds_tiff = rasterio.open(input_tiff)
srs_tiff = ds_tiff.crs

In [5]:
# Open the shapefile and get its spatial reference
ds_shp = fiona.open(input_bbox)
lyr_shp = ds_shp[0]
srs_shp = ds_shp.crs

In [15]:
def get_polygon_corners(poly):
    """Extracts the corners of a polygon"""
    coords = list(poly.exterior.coords)
    return {coord: [] for coord in coords}

def get_corners_and_edges(shapefile_path):
    # Read in the shapefile as a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)

    # Extract the corners of each polygon
    corners_dict = {}
    for i, row in gdf.iterrows():
        poly_corners = get_polygon_corners(row.geometry)
        corners_dict.update(poly_corners)

    # Create edges between adjacent corners
    for i, row in gdf.iterrows():
        coords = list(row.geometry.exterior.coords)
        for i, coord in enumerate(coords):
            next_coord = coords[(i + 1) % len(coords)]
            if coord != next_coord:
                if next_coord not in corners_dict[coord]:
                    corners_dict[coord].append(next_coord)
                if coord not in corners_dict[next_coord]:
                    corners_dict[next_coord].append(coord)

    return corners_dict

# Define the math operation you want to apply to the coordinates
def modify_coords(sbbox, x, y, sz=255):
    minx, miny, maxx, maxy = sbbox.total_bounds
    new_x = ((sz*(x - minx))/(maxx-minx))
    new_y = sz-((sz*(y - miny))/(maxy-miny)) # flip y coordinates to match annotation in HEAT paper
    
    return (new_x, new_y)

In [27]:
# Define the target size
sz=300
target_size = (sz, sz)

# Loop through each feature in the shapefile and clip the TIFF file
for feat in tqdm(ds_shp):
    base_name = feat['id']
    geom = shape(feat['geometry'])
    xmin, ymin, xmax, ymax = geom.bounds
    ##### CRS AND GEOMETRY
    # Save bbox to txt
    # print("Saving "+str(base_name))
    # print(str(srs_shp) + "\n" + str(xmin)+ str(xmax)+ str(ymin)+ str(ymax))
    with open(f"{output_folder}/geom_txt/{base_name}.txt", 'w') as f:
        f.write(str(srs_shp) + "\n" + str(xmin)+ "," + str(xmax)+ "," + str(ymin)+ "," + str(ymax))
    
    ##### CLIPPING AND RESIZING IMAGE
    # Define the output file name based on the feature ID
    clip_tiff = f"{output_folder}/clipped/{base_name}.tif"

    # print("Clipping "+str(base_name))
    # Clip the new raster to the feature polygon
    clipped_data = ds_tiff.read(window=from_bounds(xmin, ymin, xmax, ymax, ds_tiff.transform))
    window = Window(col_off=xmin, row_off=ymin, width=xmax-xmin, height=ymax-ymin)

    # Get the metadata of the original raster
    out_meta = ds_tiff.meta.copy()
    
    # Update the metadata with the new dimensions and transform
    out_meta.update({
        "driver": "GTiff",
        "height": clipped_data.shape[1],
        "width": clipped_data.shape[2],
        "transform": ds_tiff.window_transform(window)
    })
    with rasterio.open(clip_tiff, "w", **out_meta) as dest:
        dest.write(clipped_data)
    
    # Resizing
    # print("Resizing "+str(base_name))
    # Define the output file name based on the feature ID
    resize_tiff = f"{output_folder}/rgb/{base_name}.jpg"
    with Image.open(clip_tiff) as img:
        # Resize the image
        resized_img = img.resize(target_size)
        # Convert the image to JPEG format
        resized_img = resized_img.convert("RGB")

        # Save the image in JPEG format
        resized_img.save(resize_tiff, "JPEG")

    ##### INNER PLANES
    ### Selecting planes within buffer
    # Read in the shapefiles
    inner_planes = gpd.read_file(input_planes)
    bounding_box = gpd.read_file(input_bbox)
    curr_poly = bounding_box[bounding_box['FID1']==int(base_name)]
    # Select all polygons that are completely contained within the bounding box
    # print("Selecting planes "+str(base_name))
    sel_inner_planes = inner_planes[inner_planes.within(curr_poly.loc[int(base_name), 'geometry'])]
    # Write the selected polygons to a new shapefile
    out_inner_planes = f"{output_folder}/inner_planes/{base_name}.shp"
    sel_inner_planes.to_file(out_inner_planes)
    
    ### Outputting graphs to dictionary
    # print("Getting corners and edges "+str(base_name))
    edges= get_corners_and_edges(out_inner_planes)
    newedges = {}
    for key, value in edges.items():
        x, y = key
        new_key = (x, y)
        new_value = []
        for coord in value:
            new_x, new_y = coord
            new_value.append((new_x, new_y))
        newedges[new_key] = new_value
    # print(newedges)
    # print("Finalizing "+str(base_name))
    output_dict = {}
    for key, value in newedges.items():
        new_key = modify_coords(curr_poly,*key,(sz-1))
        new_v = []
             new_v.append(array(modify_coords(curr_poly,*point,(sz-1))))
        new_value = new_v
        output_dict[new_key] = new_value
    # print("Exporting NPY "+str(base_name))
    # with open(f"{output_folder}/npy_1/{base_name}.npy", 'wb') as f:
    #     pickle.dump(output_dict, f)
    f_out = f"{output_folder}/annot/{base_name}.npy"
    np.save(f_out, output_dict)
        
# Clean up
ds_tiff = None
ds_shp = None

  0%|          | 0/2465 [00:00<?, ?it/s]

In [28]:
import glob
all_annot = glob.glob("2.TrainingPreparation/EnschedeInference/annot/*")
output_folder = "2.TrainingPreparation/EnschedeInference/det_final"
for i in range(len(all_annot)):
    # print(all_annot[i])
    base_name = re.split('[/.]',all_annot[i])[4]
    annot = np.load(all_annot[i], allow_pickle=True, encoding='bytes').tolist()
    annot_list = list(np.around(list(annot.keys()),decimals=1))
    swapped_annot_list = []
    for x, y in annot_list:
        swapped_annot_list.append((y, x))
    det_out = f"{output_folder}/{base_name}.npy"
    np.save(det_out,  swapped_annot_list)